# Library Resource Management Software

In [99]:
!pip install pandas

In [97]:
!pip install IPython

In [67]:
import pandas as pd
from IPython.display import Markdown, display, HTML
import getpass # getpass is  preinstalled with python
books_csv = pd.read_csv("books.csv")
books_data = list(books_csv.itertuples(index=False, name=None))
# I print the variable to make sure it's working
# books_data

In [69]:
class Book:
    """ Initializes a book with title, author and genre """
    def __init__(self, title, author, genre):
        self.title = title
        self.author = author
        self.genre = genre
        self.is_borrowed = False

    def borrow(self):
        """ Checks if the book is borrowed if not, marks the book as borrowed """
        if not self.is_borrowed:
            self.is_borrowed = True
            display(Markdown(f"### <span style='color:green'>You borrowed the book {self.title}</span>"))

    def return_book(self):
        """ When the book is returned the book is not borrowed anymore so the is_borrowed is turning into false """
        if self.is_borrowed: 
            self.is_borrowed = False
            display(Markdown(f"### <span style='color:green'>Book '{self.title}' has been returned. </span>"))

        else:
            display(Markdown(f"### <span style='color:red'>Book '{self.title}' is not currently borrowed</span>"))


    def __str__(self):
        """ Gives the status of the book according to is_borrowed. It will be used in sorting, and for members to find available books. """
        status = "Borrowed" if self.is_borrowed else "Available"
        return f"'{self.title}' by {self.author} - Genre: {self.genre} - Status: {status}"

In [71]:
class Library:
    def __init__(self, books_data):
        """ Initializes the library with a collection of books. It's a list of tuples containing book details """
        self.books = [Book(title, author, genre) for title, author, genre in books_data]

    def add_book(self, title, author, genre):
        """ Adds book to the library """
        new_book = Book(title, author, genre)
        self.books.append(new_book)
        display(Markdown(f"### <span style='color:green'>{title} by {author} added to the library</span>"))


    def remove_book(self, title):
        """ Removes a book from the library """
        for book in self.books:
            if book.title.lower() == title.lower():
                self.books.remove(book)
                display(Markdown(f"### <span style='color:red'>{title} removed from the library</span>"))
                # return to make sure that the statement will break and it won't continue to the print function
                return
        display(Markdown(f"### <span style='color:red'>{title} not found in library</span>"))


    def view_books(self):
        """ Depicts all the books in the library """
        if not self.books:
            display(Markdown(f"### <span style='color:red'>No books available in the library</span>"))
        else:
            # Initializing the book attributes list so they can be added and displayed in a dataframe later
            titles_list = []
            authors_list = []
            genres_list = []
            status_list = []
            for book in self.books:
                # append the book attributes in the lists
                titles_list.append(book.title)
                authors_list.append(book.author)
                genres_list.append(book.genre)
                status_list.append("Borrowed" if book.is_borrowed else "Available")
            # Turning the lists into a datafame
            df = pd.DataFrame({"Title": titles_list, "Author": authors_list, "Genre": genres_list, "Status": status_list})
            display(df)

    def borrow_book(self, title, user_name):
        """ When a user borrows a book. Checks if the book is already borrowed"""
        for book in self.books:
            if book.title.lower() == title.lower():
                if not book.is_borrowed:
                    book.borrow()
                    # return to make sure that loop ends here
                    return
                else:
                    display(Markdown(f"### <span style='color:red'>{book.title} is currently borrowed</span>"))
                    # same method as above
                    return
        display(Markdown(f"### <span style='color:red'>{title} not found</span>"))


    def return_book(self, title):
        """ Returns a borrowed book """
        for book in self.books:
            if book.title.lower() == title.lower():
                if book.is_borrowed:
                    book.return_book()
                else:
                    display(Markdown(f"### <span style='color:red'>{title} was not borrowed.</span>"))
                return
        display(Markdown(f"### <span style='color:red'>{title} not found in the library</span>"))


    def search_books(self, keyword):
        """ Search for books by title, author or genre """
        results = [
            book for book in self.books
            if keyword.lower() in book.title.lower()
            or keyword.lower() in book.author.lower()
            or keyword.lower() in book.genre.lower()
        ]
        if results:
            display(Markdown(f"### results for {keyword}:"))
            # Initializing the lists for the book attributes that will after become a dataframe
            titles_list = []
            authors_list = []
            genres_list = []
            status_list = []
            for book in results:
                # Adding book attributes in the lists
                titles_list.append(book.title)
                authors_list.append(book.author)
                genres_list.append(book.genre)
                status_list.append("Borrowed" if book.is_borrowed else "Available")
            # Lists to dataframe
            df = pd.DataFrame({"Title": titles_list, "Author": authors_list, "Genre": genres_list, "Status": status_list})
            display(df)
        else:
            display(Markdown(f"### <span style='color:red'>No books found for {keyword}</span>"))


    def sort_books(self, by="title"):
        """ Sort books by title, author, genre, or availability. sorts by title but this can change with author, genre """

        def bubble_sort(key):
            n = len(self.books)
            for i in range(n):
                for j in range(0, n - i - 1):
                    if key(self.books[j]) > key(self.books[j+1]):
                        self.books[j], self.books[j + 1] = self.books[j + 1], self.books[j]

        if by == "title":
            bubble_sort(key=lambda book:book.title)
        elif by == "author":
            bubble_sort(key=lambda book:book.author)
        elif by == "genre":
            bubble_sort(key=lambda book:book.genre)
        elif by == "status":
            bubble_sort(key=lambda book:book.is_borrowed)
        else:
            display(Markdown(f"### <span style='color:red'>Invalid sort criteria: {by}</span>"))

            return
        display(Markdown(f"### Books sorted by {by}:"))
        self.view_books()

    def view_borrowed_books(self):
        """ Displays all borrowed books"""
        borrowed_books = [book for book in self.books if book.is_borrowed]
        if not borrowed_books:
            display(Markdown(f"### <span style='color:red'>No books are currently borrowed.</span>"))
        else:
            display(Markdown("### -------- Borrowed books --------"))
            # Initializing the lists so they can be displayed as dataframe in the borrowed books
            titles_list = []
            authors_list = []
            for book in borrowed_books:
                # adding the attributes to the lists
                titles_list.append(book.title)
                authors_list.append(book.author)
            # Turning the lists into a dataframe
            df = pd.DataFrame({"Title": titles_list, "Author": authors_list})
            display(df)

In [73]:
class User:
    def __init__(self, name, password, role):
        """ User initialization with name and role """
        self.name = name
        self.role = role
        self.password = password
        self.borrowed_books = []

    def authenticate(self, input_password):
        """ Checks if passwords match """
        return self.password == input_password

    def borrow_book(self, library, title):
        """ When user is borrowing a book from the library """
        for book in library.books:
            if book.title.lower() == title.lower():
                if book in self.borrowed_books:
                    display(Markdown(f"### <span style='color:red'>You have already borrowed {title}</span>"))
                    return
                elif not book.is_borrowed:
                    library.borrow_book(title, self.name)
                    self.borrowed_books.append(book)
                    return
                else:
                    display(Markdown(f"### <span style='color:red'>{title} is currently borrowed or unavailable.</span>"))
                    return
        display(Markdown(f"### <span style='color:red'>{title} not found in the library</span>"))

                    

    def return_book(self, library, title):
        """ Returns a book to the library """
        for book in self.borrowed_books:
            if book.title.lower() == title.lower():
                library.return_book(title)
                self.borrowed_books.remove(book)
                return
        display(Markdown(f"### <span style='color:red'>You don't have {title} to return</span>"))


    def view_borrowed_books(self):
        if not self.borrowed_books:
            display(Markdown(f"### <span style='color:red'>You have not borrowed any books.</span>"))
        else:
            display(Markdown("### Books you have borrowed: "))
            for book in self.borrowed_books:
                display(Markdown(f"#### {book.title} by {book.author}"))

In [74]:
class Admin(User):
    def __init__(self, name, password):
        """ Admin user initialization """
        super().__init__(name, role="admin", password=password)

    def add_book(self, library, title, author, genre):
        """ Add a new book to the library """
        library.add_book(title, author, genre)

    def remove_book(self, library, title):
        """ Remove a book from the library by title """
        library.remove_book(title)

    def remove_books_by_genre(self, library, genre):
        """ Using recursion to remove all boks of a specific genre from the library """
        if not any(book.genre.lower() == genre.lower() for book in library.books):
            display(Markdown(f"### <span style='color:green'>All books of genre {genre} have been removed.</span>"))
            return
        for book in library.books:
            # lower the input to make sure that the genre input and the genre are the same, unless the user has not typed the genre well
            if book.genre.lower() == genre.lower():
                display(Markdown(f"### <span style='color:green'>Removing {book.title} by {book.author}...</span>"))
                library.remove_book(book.title)
                break
        self.remove_books_by_genre(library, genre)

    def generate_reports(self, library):
        """ Generates reports on library usage """
        display(Markdown("### Library reports:\n"))
        library.view_borrowed_books()

In [76]:
class Member(User):
    """ Member user initialization """
    def __init__(self, name, password):
        super().__init__(name, role="member", password=password)

In [79]:
def initialize_system():
    """
    Initializes the library system with books, admin, and some members.
    """
    # Add the books to the library class so the library can be initialized
    library = Library(books_data)

    # Create a random admin user so someone can login as an admin
    admin = Admin(name="Librarian", password="admin123")
    users = {"admin": admin}

    # Add some members to make sure there are some members
    names = ["alice", "bob", "konstantinos", "george", "nick", "john", "jake", "nate", "barbara", "anna", "kate", "mark", "sara", "sasha"]
    for name in names:
        users[name] = Member(name=name, password=f"{name}123")
        
    # Add books to different people so it looks like the library has worked
    users["alice"].borrow_book(library, "1984")  
    users["bob"].borrow_book(library, "The Great Gatsby")  
    users["george"].borrow_book(library, "To Kill a Mockingbird")  


    return library, users

In [81]:
def main_menu(library, users):
    """ Main menu of the library system """
    display(Markdown("# Welcome to our library!"))

    while True:
        try:
            # prompts the user for an input
            display(Markdown("#### 1. Log in"))
            display(Markdown("#### 2. Create Account"))
            display(Markdown("#### 3. Exit"))
            choice = int(input("Enter your choice: "))
            # Make sure that the user types something between 1 and 3
            if choice > 3 or choice < 1:
                display(Markdown(f"### <span style='color:red'>Invalid choice choose a number between 1 and 3</span>"))
            else:
                if choice == 1:
                    login(users, library)
                elif choice == 2:
                    create_account(users)
                elif choice == 3:
                    display(Markdown("#### Exiting..."))
                    display(Markdown("#### Goodbye!"))
                    break
        # If the user does not type an integer catches the exception
        except ValueError:
            display(Markdown(f"### <span style='color:red'>Invalid choice type a number beween 1 and 3</span>"))
        # This is more for the development of the librry, in case something else does not work to find out what is not working
        except Exception as e:
            display(Markdown(f"### <span style='color:red'>Error occured: {e}</span>"))

            


def login(users, library):
    """ User menu when someone logins """
    display(Markdown("## ----------------------- Login Page -----------------------"))
    try:
        username = str(input("Enter username: ")).lower()
        if username in users:
            password = getpass.getpass("Enter password: ")
            user = users[username]
            if user.authenticate(password):
                display(Markdown(f"### ----------------------- Welcome back, {user.name.capitalize()} -----------------------"))
                if user.role == "admin":
                    admin_menu(user, library)
                else:
                    member_menu(user, library)
            else:
                display(Markdown(f"### <span style='color:red'>Incorrect password try again</span>"))
        else:
            raise ValueError("Invalid username please try again")
    # In case user does not type an integer
    except ValueError as e:
        display(Markdown(f"### <span style='color:red'>{e}</span>"))
    # Same as before making sure in the development phase that everything works well and if not to find what is not going good
    except Exception:
        display(Markdown(f"### <span style='color:red'>Unexpected error: {e}</span>"))



def create_account(users):
    """ Function for the account creation """
    while True:
        display(Markdown("### ------ New user! ------"))
        username = input("Username: ").lower()
        # Catch the case that the user leaves the field empty or adds only spaces
        if not username or (len(username.strip()) == 0):
            display(Markdown(f"### <span style='color:red'>Your username can't be empty</span>"))
        else:
            if username in users:
                display(Markdown(f"### <span style='color:red'>Username already exists</span>"))
            else:
                name = input("Name: ")
                # I use the same method as the username to make sure name is not empty or filled with spaces
                if not name or (len(name.strip()) == 0):
                    display(Markdown(f"### <span style='color:red'>Your name can't be empty</span>"))
                else:
                    try:
                        # If the user types admin or member in caps making sure that the program will understand the input
                        role = str(input("Enter role (admin or member): ")).lower()
                        # Make sure that the role is either admin or member
                        if role not in ["admin", "member"]:
                            raise ValueError("Invalid role try again (role should be admin or member)")
                        else:
                            password = getpass.getpass("Password: ")
                            if not password or (len(password.strip()) == 0):
                                display(Markdown(f"### <span style='color:red'>Your password can't be empty</span>"))
                            else:
                                if role == "admin":
                                    # make an instance of the new admin and save
                                    users[username] = Admin(name=name, password=password)
                                    display(Markdown(f"### <span style='color:green'>Admin account created for {name}!</span>"))
                                    break
                                elif role == "member":
                                    # same method as for the admin, but here is for member
                                    users[username] = Member(name=name, password=password)
                                    display(Markdown(f"### <span style='color:green'>Member account created for {name}!</span>"))
                                    break
                    except ValueError as e:
                        display(Markdown(f"### <span style='color:red'>{e}</span>"))

def admin_menu(admin, library):
    """ Management of the library by admin """
    while True:
        try:
            display(Markdown("### ---- Admin Menu ---- "))
            display(Markdown("#### 1. Search for a Book"))
            display(Markdown("#### 2. Add Book"))
            display(Markdown("#### 3. Remove Book"))
            display(Markdown("#### 4. Remove books by genre"))
            display(Markdown("#### 5. View Books"))
            display(Markdown("#### 6. Generate Reports"))
            display(Markdown("#### 7. Log Out"))
            choice = int(input("What do you need today? "))
            # Make sure that the user will input a valid option
            if choice < 1 or choice > 7:
                raise ValueError
            if choice == 1:
                keyword = input("Enter the title, genre, or the author of the book: ")
                library.search_books(keyword)
            elif choice == 2:
                title = input("book title: ")
                author = input("book author: ")
                genre = input("book genre: ")
                admin.add_book(library, title, author, genre)
            elif choice == 3:
                title = input("Which title to remove? ")
                admin.remove_book(library, title)
            elif choice == 4:
                genre = input("Which genre to remove? ")
                admin.remove_books_by_genre(library, genre)
            elif choice == 5:
                by = input("Display books by title, author, genre, or status? ")
                library.sort_books(by)
            elif choice == 6:
                admin.generate_reports(library)
            elif choice == 7:
                display(Markdown(f"#### <span style='color:blue'>Logging out...</span>"))
                break
            else:
                display(Markdown(f"### <span style='color:red'>Invalid Choice: Enter a number between 1 and 7</span>"))        # Catch the case that user types anything but an int
        except ValueError:
            display(Markdown(f"### <span style='color:red'>Invalid Choice: Enter a number between 1 and 7</span>"))

        
                        
def member_menu(member, library):
    """ Member menu """
    while True:
        try:
            display(Markdown("### ---- Main Menu ---- "))
            display(Markdown("#### 1. Search for a Book"))
            display(Markdown("#### 2. View Books"))
            display(Markdown("#### 3. Borrow Book"))
            display(Markdown("#### 4. Return Book"))
            display(Markdown("#### 5. View Borrowed Books"))
            display(Markdown("#### 6. Log Out"))
            # Make sure that the choice will be an integer
            choice = int(input("Enter your choice: "))
            if choice < 1 or choice > 6:
                display(Markdown(f"### <span style='color:red'>Invalid Choice: Enter a number between 1 and 6</span>"))
            if choice == 1:
                keyword = input("Enter the title, genre, or the author of the book: ")
                library.search_books(keyword)
            elif choice == 2:
                by = input("View books by title, genre, author or status? ")
                library.sort_books(by)
            elif choice == 3:
                title = input("Enter the book title to borrow: ")
                member.borrow_book(library, title)
            elif choice == 4:
                title = input("Enter the book title to return: ")
                member.return_book(library, title)
            elif choice == 5:
                member.view_borrowed_books()
            elif choice == 6:
                display(Markdown(f"#### <span style='color:blue'>Hope to see you again!</span>"))
                display(Markdown(f"#### <span style='color:blue'>Logging out...</span>"))
                break
        except ValueError:
            display(Markdown(f"### <span style='color:red'>Invalid Choice: Enter a number between 1 and 6</span>"))

In [101]:
while True:
    # I give two options for the running of the library. First option the library starts from zero without anything inside
    # Second option the initialize system runs and the library takes some users, an admin and some books to look like it has already worked
    try:
        display(Markdown("### Type 1 if you want to initalize the library with some books and users, or "
                         "type 2 if you want to start with an empty library."))
        initialize_choice = int(input("Enter your choice: "))
        # make sure that the choice will be either 1 or 2
        if initialize_choice < 1 or initialize_choice > 2:
            display(Markdown(f"### <span style='color:red'>Type a number between 1 and 2! Type 1 for library with pre-existing books and users, or type 2 for an empty library</span>"))
        elif initialize_choice == 1:
            library, users = initialize_system()
            break
        elif initialize_choice == 2:
            library = Library([])
            users = {}
            break
    # Catch the case that the user types anything but an integer
    except ValueError:
        display(Markdown(f"### <span style='color:red'>Type a number between 1 and 2! Type 1 for library with pre-existing books and users, or type 2 for an empty library</span>"))

main_menu(library, users)